In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

vec = cPickle.load(open('../../preprocess/allfields_with_embedding_19995.p', 'rb'))
cohen_vec = cPickle.load(open('../../preprocess/cohendata_dedup_19995.p'))
#da_vec = cPickle.load(open('../data/vectorizers/decision_aids_vec_5000.p'))

Using TensorFlow backend.


[]


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1
    
H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [3]:
index = vec.index['abstract']
vec.X = vec.X[index[0]:index[1]]
train_X = vec.X
train_X = map(lambda s : [vec.idx2word[t] for t in s if t > 1], train_X)

In [5]:
test_X = cohen_vec.X
test_X = map(lambda s : [vec.idx2word[t] for t in s if t > 1], test_X)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False)
train_TF = tfidf.fit_transform(train_X)

In [6]:
test_TF = tfidf.transform(test_X)
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
from sklearn.preprocessing import normalize
# embedds_n = normalize(np.array(test_TF.todense()), 'l2')
# scores = np.dot(embedds_n, embedds_n.T)
scores = cosine_similarity(test_TF)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [8]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.DataFrame(pd.Series(rocs))

0.790149201688
                                          0
ACEInhibitors_processed            0.809715
ADHD_processed                     0.896114
Antihistamines_processed           0.810739
AtypicalAntipsychotics_processed   0.746003
BetaBlockers_processed             0.672498
CalciumChannelBlockers_processed   0.665070
Estrogens_processed                0.874923
NSAIDS_processed                   0.848748
Opiods_processed                   0.812200
OralHypoglycemics_processed        0.786787
ProtonPumpInhibitors_processed     0.809734
SkeletalMuscleRelaxants_processed  0.596118
Statins_processed                  0.788765
Triptans_processed                 0.924097


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(tokenizer=lambda i:i, lowercase=False)
train_C = count.fit_transform(train_X)
test_C = count.transform(test_X)

In [38]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
best_mean = 0.0
for n_top in range(1, 30) :
    from sklearn.decomposition import LatentDirichletAllocation
    lda = LatentDirichletAllocation(n_components=n_top)
    train_lda = lda.fit_transform(train_C)
    test_lda = lda.transform(test_C)
    from sklearn.preprocessing import normalize
    embedds_n = normalize(test_lda, 'l2')
    scores = np.dot(embedds_n, embedds_n.T)
    scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
    scores = np.array(scores)
    from sklearn.metrics import roc_auc_score
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H[i], scores[i])
    print n_top, np.mean(aucs)

1 0.500310279253
2 0.623527038882
3 0.595615915264
4 0.640634834028
5 0.645601735729
6 0.709009958285
7 0.732644507308
8 0.72626853244
9 0.705524543521
10 0.698009737851
11 0.678954949293
12 0.699571704167
13 0.671752370888
14 0.688309078422
15 0.673867209576
16 0.67140868851
17 0.67585329223
18 0.673402155916
19 0.650542116128
20 0.673550168861


KeyboardInterrupt: 

In [14]:
#10 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.671772112187
ACEInhibitors_processed              0.676719
ADHD_processed                       0.768559
Antihistamines_processed             0.642429
AtypicalAntipsychotics_processed     0.703576
BetaBlockers_processed               0.610434
CalciumChannelBlockers_processed     0.629003
Estrogens_processed                  0.600515
NSAIDS_processed                     0.645389
Opiods_processed                     0.735651
OralHypoglycemics_processed          0.655636
ProtonPumpInhibitors_processed       0.648496
SkeletalMuscleRelaxants_processed    0.760118
Statins_processed                    0.661097
Triptans_processed                   0.764934
dtype: float64


In [19]:
#LDA 20
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.DataFrame(pd.Series(rocs))

0.663811713035
                                          0
ACEInhibitors_processed            0.570558
ADHD_processed                     0.759764
Antihistamines_processed           0.741349
AtypicalAntipsychotics_processed   0.728739
BetaBlockers_processed             0.596694
CalciumChannelBlockers_processed   0.554335
Estrogens_processed                0.596897
NSAIDS_processed                   0.696318
Opiods_processed                   0.635425
OralHypoglycemics_processed        0.749142
ProtonPumpInhibitors_processed     0.681956
SkeletalMuscleRelaxants_processed  0.696130
Statins_processed                  0.616788
Triptans_processed                 0.719855


In [24]:
#LDA 30
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.DataFrame(pd.Series(rocs))

0.649504072906
                                          0
ACEInhibitors_processed            0.626700
ADHD_processed                     0.785363
Antihistamines_processed           0.680185
AtypicalAntipsychotics_processed   0.668366
BetaBlockers_processed             0.614333
CalciumChannelBlockers_processed   0.590059
Estrogens_processed                0.582509
NSAIDS_processed                   0.665451
Opiods_processed                   0.605092
OralHypoglycemics_processed        0.682193
ProtonPumpInhibitors_processed     0.588913
SkeletalMuscleRelaxants_processed  0.667337
Statins_processed                  0.579902
Triptans_processed                 0.786799


In [29]:
#LDA 5
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.DataFrame(pd.Series(rocs))

0.688458054695
                                          0
ACEInhibitors_processed            0.733466
ADHD_processed                     0.784954
Antihistamines_processed           0.765961
AtypicalAntipsychotics_processed   0.749551
BetaBlockers_processed             0.561474
CalciumChannelBlockers_processed   0.606236
Estrogens_processed                0.595585
NSAIDS_processed                   0.683773
Opiods_processed                   0.726598
OralHypoglycemics_processed        0.750995
ProtonPumpInhibitors_processed     0.619063
SkeletalMuscleRelaxants_processed  0.704253
Statins_processed                  0.734470
Triptans_processed                 0.745464


In [11]:
#15 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.632397412097
ACEInhibitors_processed              0.603385
ADHD_processed                       0.792231
Antihistamines_processed             0.725511
AtypicalAntipsychotics_processed     0.729733
BetaBlockers_processed               0.583911
CalciumChannelBlockers_processed     0.534427
Estrogens_processed                  0.549480
NSAIDS_processed                     0.580660
Opiods_processed                     0.614504
OralHypoglycemics_processed          0.645920
ProtonPumpInhibitors_processed       0.609919
SkeletalMuscleRelaxants_processed    0.739308
Statins_processed                    0.571358
Triptans_processed                   0.678575
dtype: float64


In [16]:
#200 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.630408035104
ACEInhibitors_processed              0.568822
ADHD_processed                       0.745592
Antihistamines_processed             0.648518
AtypicalAntipsychotics_processed     0.631148
BetaBlockers_processed               0.571659
CalciumChannelBlockers_processed     0.535410
Estrogens_processed                  0.554959
NSAIDS_processed                     0.642947
Opiods_processed                     0.615354
OralHypoglycemics_processed          0.637152
ProtonPumpInhibitors_processed       0.654760
SkeletalMuscleRelaxants_processed    0.644289
Statins_processed                    0.619789
Triptans_processed                   0.752280
dtype: float64


In [21]:
#4 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.679566777892
ACEInhibitors_processed              0.635814
ADHD_processed                       0.820726
Antihistamines_processed             0.638396
AtypicalAntipsychotics_processed     0.842678
BetaBlockers_processed               0.552658
CalciumChannelBlockers_processed     0.589661
Estrogens_processed                  0.680963
NSAIDS_processed                     0.638793
Opiods_processed                     0.593138
OralHypoglycemics_processed          0.780940
ProtonPumpInhibitors_processed       0.745649
SkeletalMuscleRelaxants_processed    0.613927
Statins_processed                    0.584975
Triptans_processed                   0.685248
dtype: float64


In [26]:
#7 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.721504580854
ACEInhibitors_processed              0.720642
ADHD_processed                       0.800462
Antihistamines_processed             0.700825
AtypicalAntipsychotics_processed     0.725342
BetaBlockers_processed               0.673189
CalciumChannelBlockers_processed     0.651937
Estrogens_processed                  0.643024
NSAIDS_processed                     0.730007
Opiods_processed                     0.699040
OralHypoglycemics_processed          0.834619
ProtonPumpInhibitors_processed       0.713034
SkeletalMuscleRelaxants_processed    0.688588
Statins_processed                    0.694711
Triptans_processed                   0.790590
dtype: float64


In [31]:
#8 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.700454416337
ACEInhibitors_processed              0.684900
ADHD_processed                       0.820319
Antihistamines_processed             0.779867
AtypicalAntipsychotics_processed     0.750762
BetaBlockers_processed               0.603489
CalciumChannelBlockers_processed     0.587012
Estrogens_processed                  0.630975
NSAIDS_processed                     0.712957
Opiods_processed                     0.646028
OralHypoglycemics_processed          0.785506
ProtonPumpInhibitors_processed       0.690359
SkeletalMuscleRelaxants_processed    0.752506
Statins_processed                    0.679658
Triptans_processed                   0.760451
dtype: float64


In [36]:
#6 LDA
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print pd.Series(rocs)

0.727974845604
ACEInhibitors_processed              0.728741
ADHD_processed                       0.789238
Antihistamines_processed             0.782176
AtypicalAntipsychotics_processed     0.700299
BetaBlockers_processed               0.563129
CalciumChannelBlockers_processed     0.621448
Estrogens_processed                  0.668034
NSAIDS_processed                     0.732668
Opiods_processed                     0.753121
OralHypoglycemics_processed          0.873091
ProtonPumpInhibitors_processed       0.737209
SkeletalMuscleRelaxants_processed    0.681528
Statins_processed                    0.804431
Triptans_processed                   0.814155
dtype: float64
